In [5]:
import os
from pathlib import Path
DIR_HOME = Path(os.getcwd()).parent
DIR_CONVERSATION = DIR_HOME / "data" / "conversations"

import sys
sys.path.append(str(DIR_HOME))

import json
import pandas as pd
with open(DIR_CONVERSATION / "text-davinci-003-single-response.json") as f:
    responses = json.load(f)

import sqlalchemy
import mysql.connector as connection

from src.utils import cohen_d, norm_diff_stdev
from src.metrics import SentenceBERTDiversity, Length
grp_metrics = [SentenceBERTDiversity("paraphrase-MiniLM-L3-v2")]
ind_metrics = [Length()]

In [2]:
for response in responses:
    for metric in grp_metrics:
        response[metric.name] = metric(response["completion"])[0]
    for metric in ind_metrics:
        response[metric.name] = metric(response["completion"])

df_responses = pd.DataFrame(responses).set_index(["qid", "cid"])
df_responses.head()

model  max_tokens                    stop  n  temperature  \
qid cid                                                                         
0   0    text-davinci-003         128  [Therapist:, Patient:]  8          0.5   
    1    text-davinci-003         128  [Therapist:, Patient:]  8          0.1   
    2    text-davinci-003         128  [Therapist:, Patient:]  8          1.0   
    3    text-davinci-003         128  [Therapist:, Patient:]  8          0.5   
    4    text-davinci-003         128  [Therapist:, Patient:]  8          0.5   

         top_p  frequency_penalty  presence_penalty  \
qid cid                                               
0   0      0.7                1.0               1.0   
    1      0.7                1.0               1.0   
    2      0.7                1.0               1.0   
    3      0.5                1.0               1.0   
    4      0.9                1.0               1.0   

                                                    prompt  \
qid cid                                                      
0   0    Below is a conversation between a patient and ...   
    1    Below is a conversation between a patient and ...   
    2    Below is a conversation between a patient and ...   
    3    Below is a conversation between a patient and ...   
    4    Below is a conversation between a patient and ...   

                                                 question  \
qid cid                                                     
0   0    How would you feel if someone called you a jerk?   
    1    How would you feel if someone called you a jerk?   
    2    How would you feel if someone called you a jerk?   
    3    How would you feel if someone called you a jerk?   
    4    How would you feel if someone called you a jerk?   

                                                completion  \
qid cid                                                      
0   0    [I would feel embarrassed and ashamed. I would...   
    1    [I would feel very hurt and angry. I would als...   
    2    [I would feel very upset and angry. I would al...   
    3    [I would feel very hurt and disrespected. I th...   
    4    [I would feel very hurt and angry. I would als...   

         sentencebert_diversity                            length  
qid cid                                                            
0   0                  0.254657  [39, 41, 41, 41, 44, 42, 31, 43]  
    1                  0.166667  [33, 32, 30, 33, 41, 30, 33, 33]  
    2                  0.309026  [36, 44, 45, 43, 44, 45, 33, 55]  
    3                  0.342677  [44, 46, 42, 29, 41, 40, 42, 37]  
    4                  0.311913  [32, 43, 45, 47, 40, 42, 52, 39]

In [3]:
base_cid = 0
last_cid = 10
last_qid = 23

grp_stats = []
ind_stats = []

for cid in range((base_cid + 1), (last_cid + 1)):

    for metric in grp_metrics:
        val_base = df_responses.xs(base_cid, level="cid").loc[:, metric.name].values
        val_test = df_responses.xs(cid, level="cid").loc[:, metric.name].values
        grp_stats.append({"base_cfg": base_cid,
                          "test_cfg": cid,
                          "metric": metric.name,
                          "cohen_d": f"{cohen_d(val_base, val_test):.3f}",
                          "norm_diff_stdev": f"{norm_diff_stdev(val_base, val_test):.3f}"})

    for qid in range(last_qid + 1):
        for metric in ind_metrics:
            val_base = df_responses.xs((qid, base_cid), level=["qid", "cid"]).loc[:, metric.name].values[0]
            val_test = df_responses.xs((qid, cid), level=["qid", "cid"]).loc[:, metric.name].values[0]
            ind_stats.append({"base_cfg": base_cid,
                              "test_cfg": cid,
                              "qid": qid,
                              "metric": metric.name,
                              "cohen_d": f"{cohen_d(val_base, val_test):.3f}",
                              "norm_diff_stdev": f"{norm_diff_stdev(val_base, val_test):.3f}"})

In [4]:
pd.DataFrame(grp_stats)

,base_cfg,test_cfg,metric,cohen_d,norm_diff_stdev
0,0,1,sentencebert_diversity,1.584,0.416
1,0,2,sentencebert_diversity,-0.128,0.167
2,0,3,sentencebert_diversity,0.176,0.099
3,0,4,sentencebert_diversity,0.073,0.055
4,0,5,sentencebert_diversity,0.321,0.155
5,0,6,sentencebert_diversity,0.290,0.194
6,0,7,sentencebert_diversity,0.018,0.006
7,0,8,sentencebert_diversity,0.209,0.030
8,0,9,sentencebert_diversity,0.150,0.194
9,0,10,sentencebert_diversity,0.050,0.058


In [5]:
pd.DataFrame(ind_stats)

,base_cfg,test_cfg,qid,metric,cohen_d,norm_diff_stdev
0,0,1,0,length,1.902,0.156
1,0,1,1,length,1.007,0.070
2,0,1,2,length,-0.063,0.521
3,0,1,3,length,0.238,1.137
4,0,1,4,length,-0.548,0.360
...,...,...,...,...,...,...
235,0,10,19,length,0.568,-0.913
236,0,10,20,length,0.025,0.720
237,0,10,21,length,1.059,0.180
238,0,10,22,length,-1.256,-0.225


## __Compute LIWC2015 Metrics with `DLATK`__

In [39]:
table_name = "text_davinci_003_single_response"
df_responses = pd.DataFrame(responses) \
                    .rename(columns={"completion": "message"}) \
                    .loc[:, ["qid", "cid", "message"]] \
                    .explode("message")

sql_drop_table = f"""DROP TABLE IF EXISTS {table_name};"""
sql_create_table = f"""
CREATE TABLE {table_name} (
    message_id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY,
    qid INT(10) UNSIGNED NOT NULL,
    cid INT(10) UNSIGNED NOT NULL,
    message TEXT
);
"""

In [40]:
cnx = connection.connect(host="localhost",
                         database="wangy49",
                         user="wangy49")
cursor = cnx.cursor()

# Create a SQLAlchemy engine from the MySQL connection
engine = sqlalchemy.create_engine('mysql+mysqlconnector://', creator=lambda: cnx)

In [41]:
cursor.execute(sql_drop_table)
cursor.execute(sql_create_table)
cnx.commit()

In [42]:
df_responses.to_sql(table_name, 
                    con=engine, 
                    if_exists="append", 
                    index=False)

2112

In [43]:
cnx.close()